In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import re

# 1. 웹드라이버 접속
driver = webdriver.Chrome()

# 2. 데이터 불러오기
df = pd.read_excel('1122_naver_info.xlsx')

# 3. 정보 수집 준비
reviews_data = []  # 리뷰 정보를 저장할 리스트

# 4. 리뷰 URL 접속 및 '더보기' 클릭
for url in df['Review URL']:  # 첫 번째부터 다섯 번째 행까지만 순회
    try:
        # URL에 접속
        driver.get(url)
        time.sleep(5)  # 웹 페이지 로드를 위한 대기 시간

        review_counter = 0  # 현재 URL의 리뷰 카운터 초기화

        # 더보기 버튼이 모두 클릭될 때까지 반복
        while True:
            try:
                more_button = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > div.NSTUp > div > a')
                driver.execute_script("arguments[0].click();", more_button)
                time.sleep(2)  # '더보기' 버튼 클릭 후 로딩 시간
            except NoSuchElementException:
                break  # 더 이상 '더보기' 버튼이 없으면 반복 중단
        
        time.sleep(3) 

        # 5. 리뷰 데이터 파싱
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        time.sleep(2)
        
        # 방문자 리뷰 개수 가져오기
        selector = '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > h2 > span.place_section_count'
        review_count_element = driver.find_element(By.CSS_SELECTOR, selector)
        review_count_text = review_count_element.text if review_count_element else '0'
        total_review_count = int(re.findall(r'\d+', review_count_text)[0]) if review_count_text.isdigit() else 0


        # 리뷰 데이터 가져오기
        for li_index in range(1, total_review_count + 1):
            # 최대 리뷰 개수 확인
            if review_counter >= 50:
                break  # 최대 리뷰 개수에 도달하면 루프 종료

            review_xpath_base = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{li_index}]'
            review_xpath_with_photo = f'{review_xpath_base}/div[3]/a'
            review_xpath_without_photo = f'{review_xpath_base}/div[2]/a'
            
            review_text = "No review found"  # 기본값 설정
            try:
                elements = driver.find_elements(By.XPATH, review_xpath_without_photo)
                if elements:
                    review_text = elements[0].text
                else:
                    elements = driver.find_elements(By.XPATH, review_xpath_with_photo)
                    if elements:
                        review_text = elements[0].text
            except NoSuchElementException:
                continue  # 해당 인덱스에 리뷰가 없는 경우

            # 각 리뷰 정보를 리스트에 추가
            reviews_data.append({
                'Review URL': url,
                'Total Review Count': total_review_count,
                'Review Index': li_index,
                'Review Text': review_text
            })
            review_counter += 1  # 리뷰 카운터 증가
            time.sleep(0.1)
    except Exception as e:
        print(f'Error while processing {url}: {e}')

# 7. 데이터 저장
reviews_df = pd.DataFrame(reviews_data)
reviews_df.to_excel('dede.xlsx', index=False)

# 웹드라이버 종료
driver.quit()

Error while processing https://pcmap.place.naver.com/place/1226977800/review/visitor: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > h2 > span.place_section_count"}
  (Session info: chrome=119.0.6045.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7AC7082B2+55298]
	(No symbol) [0x00007FF7AC675E02]
	(No symbol) [0x00007FF7AC5305AB]
	(No symbol) [0x00007FF7AC57175C]
	(No symbol) [0x00007FF7AC5718DC]
	(No symbol) [0x00007FF7AC5ACBC7]
	(No symbol) [0x00007FF7AC5920EF]
	(No symbol) [0x00007FF7AC5AAAA4]
	(No symbol) [0x00007FF7AC591E83]
	(No symbol) [0x00007FF7AC56670A]
	(No symbol) [0x00007FF7AC567964]
	GetHandleVerifier [0x00007FF7ACA80AAB+3694587]
	GetHandleVerifier [0x00007FF7ACAD728E+4048862]
	GetHandleVerifier 